# Guided Filter and NLM Denoising

In [1]:
import cv2
import numpy as np
import os
from pathlib import Path

In [2]:
input_folder = "images"
output_folder_guided = "borui_output_guided"
output_folder_nlm = "borui_output_nlm"
process_guided = True
process_nlm = True
max_side = 1024
os.makedirs(output_folder_guided, exist_ok=True)
os.makedirs(output_folder_nlm, exist_ok=True)
image_extensions = {".jpg", ".jpeg", ".png", ".bmp", ".JPG", ".JPEG", ".PNG", ".BMP"}

In [3]:
def resize_if_needed(image, max_side):
    h, w = image.shape[:2]
    scale = max(h, w) / float(max_side)
    if scale <= 1.0:
        return image, 1.0
    new_w = int(w / scale)
    new_h = int(h / scale)
    resized = cv2.resize(image, (new_w, new_h), interpolation=cv2.INTER_AREA)
    return resized, scale


def guided_filter_fast(image, radius=4, eps=1e-2):
    src = image.astype(np.float32) / 255.0
    if src.ndim == 2:
        guide = src
    else:
        guide = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY).astype(np.float32) / 255.0
    if hasattr(cv2, "ximgproc") and hasattr(cv2.ximgproc, "guidedFilter"):
        gf = cv2.ximgproc.guidedFilter(guide=guide, src=src, radius=radius, eps=eps)
        out = (gf * 255.0).clip(0, 255).astype(np.uint8)
        return out
    img = src
    if img.ndim == 2:
        img = img[:, :, None]
    h, w, c = img.shape
    result = np.zeros_like(img, dtype=np.float32)
    ksize = (radius, radius)
    for ch in range(c):
        I = img[:, :, ch]
        mean_I = cv2.boxFilter(I, cv2.CV_32F, ksize)
        mean_II = cv2.boxFilter(I * I, cv2.CV_32F, ksize)
        var_I = mean_II - mean_I * mean_I
        a = var_I / (var_I + eps)
        b = mean_I - a * mean_I
        mean_a = cv2.boxFilter(a, cv2.CV_32F, ksize)
        mean_b = cv2.boxFilter(b, cv2.CV_32F, ksize)
        q = mean_a * I + mean_b
        result[:, :, ch] = q
    result = np.squeeze(result)
    result = np.clip(result * 255.0, 0, 255).astype(np.uint8)
    return result


def nlm_denoise_fast(image, h=7, template_window_size=5, search_window_size=13):
    if image.ndim == 3 and image.shape[2] == 3:
        denoised = cv2.fastNlMeansDenoisingColored(
            image,
            None,
            h,
            h,
            template_window_size,
            search_window_size,
        )
        return denoised
    if image.ndim == 3 and image.shape[2] == 1:
        gray = image[:, :, 0]
    else:
        gray = image
    denoised = cv2.fastNlMeansDenoising(
        gray,
        None,
        h,
        template_window_size,
        search_window_size,
    )
    if image.ndim == 3 and image.shape[2] == 3:
        denoised = cv2.cvtColor(denoised, cv2.COLOR_GRAY2BGR)
    return denoised

In [4]:
input_path = Path(input_folder)
files = [f for f in input_path.iterdir() if f.suffix in image_extensions]
print(f"found {len(files)} images")
for img_file in files:
    image = cv2.imread(str(img_file))
    if image is None:
        print(f"skip {img_file.name}")
        continue
    img_small, scale = resize_if_needed(image, max_side)
    if process_guided:
        guided_img = guided_filter_fast(img_small, radius=4, eps=1e-2)
        if scale != 1.0:
            guided_img = cv2.resize(
                guided_img,
                (image.shape[1], image.shape[0]),
                interpolation=cv2.INTER_CUBIC,
            )
        out_path_g = os.path.join(output_folder_guided, img_file.name)
        cv2.imwrite(out_path_g, guided_img)
    if process_nlm:
        nlm_img = nlm_denoise_fast(img_small, h=7, template_window_size=5, search_window_size=13)
        if scale != 1.0:
            nlm_img = cv2.resize(
                nlm_img,
                (image.shape[1], image.shape[0]),
                interpolation=cv2.INTER_CUBIC,
            )
        out_path_n = os.path.join(output_folder_nlm, img_file.name)
        cv2.imwrite(out_path_n, nlm_img)
    print(f"processed {img_file.name}")
print("done")

found 101 images
processed IMG_1862.jpg
processed IMG_1865.jpg
processed IMG_1901.jpg
processed IMG_1859.jpg
processed IMG_1906.jpg
processed IMG_1908.jpg
processed IMG_1881.jpg
processed IMG_1850.jpg
processed IMG_1851 (1).jpg
processed IMG_1886.jpg
processed IMG_1857.jpg
processed IMG_1855 (1).jpg
processed IMG_1846.jpg
processed IMG_1897.jpg
processed IMG_1893 (1).jpg
processed IMG_1897 (1).jpg
processed IMG_1841.jpg
processed IMG_1890.jpg
processed IMG_1919.jpg
processed IMG_1833.jpg
processed IMG_1917.jpg
processed IMG_1848.jpg
processed IMG_1899.jpg
processed IMG_1910.jpg
processed IMG_1848 (1).jpg
processed IMG_1834.jpg
processed IMG_1887.jpg
processed IMG_1856.jpg
processed IMG_1892 (1).jpg
processed IMG_1909.jpg
processed IMG_1896 (1).jpg
processed IMG_1851.jpg
processed IMG_1849 (1).jpg
processed IMG_1907.jpg
processed IMG_1900.jpg
processed IMG_1845 (1).jpg
processed IMG_1889.jpg
processed IMG_1858.jpg
processed IMG_1864.jpg
processed IMG_1863.jpg
processed IMG_1849.jpg
proc